Introduction to DataIngestion

In [1]:
import os
from typing import List,Dict,Any
import pandas as pd

# Understanding Document Structure in Langchain

In [2]:
# creating a simple document
from langchain_core.documents import Document
doc = Document(
    page_content="This is the main text content that will be embedded and searched",
    metadata={
        "source":"example.txt",
        "page":1,
        "author":"Venkatesh",
        "date_created":"2026-06-02",
        "custom_filed":"any_value"
    }


)


In [3]:
print("Printing Document structure")
print("=="*60)
print(f"Content :{doc.page_content}")
print(f"Metadata :{doc.metadata}")

Printing Document structure
Content :This is the main text content that will be embedded and searched
Metadata :{'source': 'example.txt', 'page': 1, 'author': 'Venkatesh', 'date_created': '2026-06-02', 'custom_filed': 'any_value'}


In [4]:
# Ingesting and Parsing Text data using Document.
import os
os.makedirs("data/textfiles",exist_ok=True)

In [5]:
sample_texts={
    "data/textfiles/python_intro.txt":"""Python is a high‑level, beginner‑friendly programming language created by Guido van Rossum in 1991. It is widely used for web development, data analysis, artificial intelligence, automation, and more because of its simple, readable syntax and powerful libraries. 

What is Python?
General-purpose language: Can be used for web apps, desktop software, scripting, data science, and machine learning.

Cross-platform: Works on Windows, macOS, Linux, and even Raspberry Pi.

Interpreted: Code runs line by line, making it easier to test and debug.

Open-source: Free to use, with a huge global community.

Why Python is Popular
Readable syntax: Its code looks close to English"""



}

In [6]:
for filepath,content in sample_texts.items():
    with open(filepath,"w",encoding="utf-8") as f:
        f.write(content)

print ("Simple Text file s created !")

Simple Text file s created !


In [7]:
from langchain_community.document_loaders import TextLoader

# Loading a single file
loader = TextLoader("data/textfiles/python_intro.txt", encoding='utf-8')
docuemnts = loader.load()

print(docuemnts)
print("Document Return type ! "  , type(docuemnts))
print(f"Loaded {len(docuemnts)} Documents")
print(f"content preview : {docuemnts[0].page_content[:100]}")
print(f"Metadata: {docuemnts[0].metadata}")

e:\RAG_KRISH\RAG_UV\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': 'data/textfiles/python_intro.txt'}, page_content='Python is a high‑level, beginner‑friendly programming language created by Guido van Rossum in 1991. It is widely used for web development, data analysis, artificial intelligence, automation, and more because of its simple, readable syntax and powerful libraries. \n\nWhat is Python?\nGeneral-purpose language: Can be used for web apps, desktop software, scripting, data science, and machine learning.\n\nCross-platform: Works on Windows, macOS, Linux, and even Raspberry Pi.\n\nInterpreted: Code runs line by line, making it easier to test and debug.\n\nOpen-source: Free to use, with a huge global community.\n\nWhy Python is Popular\nReadable syntax: Its code looks close to English')]
Document Return type !  <class 'list'>
Loaded 1 Documents
content preview : Python is a high‑level, beginner‑friendly programming language created by Guido van Rossum in 1991. 
Metadata: {'source': 'data/textfiles/python_intro.txt'}

In [8]:
# Read Text file from all the directories - Multiple text files
from langchain_community.document_loaders import DirectoryLoader

# load all the text files from the directory

dir_loader = DirectoryLoader(
    "data/textfiles",
    glob="**/*.txt" , # pattern to match text files
    loader_cls =TextLoader,
    loader_kwargs={'encoding':'utf-8'},
    show_progress=True
)

docuemnts = dir_loader.load()

print(f"Loaded {len(docuemnts)} documents")

for i , doc in enumerate(docuemnts):
    print(f"\n Document {i+1}: ")
    print("=="*60)
    print(f" Source: {doc.metadata['source']}")
    print(f"Length : {len(doc.page_content)} characters")

100%|██████████| 2/2 [00:00<00:00, 123.26it/s]

Loaded 2 documents

 Document 1: 
 Source: data\textfiles\langchain.txt
Length : 2200 characters

 Document 2: 
 Source: data\textfiles\python_intro.txt
Length : 672 characters


# Text splitting techinques

In [9]:
from langchain_text_splitters import(
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)

In [10]:
text = docuemnts[0].page_content
text

'LangChain is an open‑source framework that makes it easier to build applications powered by large language models (LLMs) like GPT‑4. It helps developers connect models with external data, tools, and workflows, enabling advanced use cases such as chatbots, retrieval‑augmented generation (RAG), and autonomous agents. \n\n🔍 What LangChain Is\nFramework for LLM apps: Provides building blocks to integrate language models with data sources, APIs, and tools.\n\nSupports Python & JavaScript: Widely accessible for developers in both ecosystems.\n\nAgent Architecture: Comes with pre‑built agent structures that can interact with multiple tools and adapt quickly.\n\nChains: Core concept where you define sequences of steps (like prompt → model → parser → output).\n\n✨ Key Features\nModular Workflow: You can chain multiple steps together for reusable pipelines.\n\nPrompt Management: Tools for prompt engineering and memory handling.\n\nIntegrations: Works with OpenAI, Anthropic, Google, Hugging Face

In [12]:
char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function = len  # how to measure the chunk size
)


char_chunks = char_splitter.split_text(text)
print(f"Len {len(char_chunks)} chunks ")
print("=="*50)
print(f"First Chunk: {char_chunks[0][:100]}...")
print("=="*50)
print(f"Second Chunk: {char_chunks[1][:100]}...")

Created a chunk of size 317, which is longer than the specified 200


Len 12 chunks 
First Chunk: LangChain is an open‑source framework that makes it easier to build applications powered by large la...
Second Chunk: 🔍 What LangChain Is
Framework for LLM apps: Provides building blocks to integrate language models wi...


In [13]:
recursive_splitter = RecursiveCharacterTextSplitter(
    #separators=["\n\n", "\n" ," ",""],
    separators=["\n"],
    chunk_size=200,
    chunk_overlap=50,
    length_function = len
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Len {len(recursive_chunks)} chunks ")
print("=="*50)
print(f"First Chunk: {recursive_chunks[0][:1000]}...")
print("=="*50)
print(f"Second Chunk: {recursive_chunks[1][:1000]}...")
print("=="*50)
print(f"Third Chunk: {recursive_chunks[2][:1000]}...")

Len 13 chunks 
First Chunk: LangChain is an open‑source framework that makes it easier to build applications powered by large language models (LLMs) like GPT‑4. It helps developers connect models with external data, tools, and workflows, enabling advanced use cases such as chatbots, retrieval‑augmented generation (RAG), and autonomous agents. ...
Second Chunk: 🔍 What LangChain Is
Framework for LLM apps: Provides building blocks to integrate language models with data sources, APIs, and tools....
Third Chunk: Supports Python & JavaScript: Widely accessible for developers in both ecosystems.

Agent Architecture: Comes with pre‑built agent structures that can interact with multiple tools and adapt quickly....
